# Python

*Refrences*

* *https://docs.python.org/3/*
* *https://github.com/taizilongxu/interview_python*
* *https://mengyangyang.org/tags/Python/*
* *流畅的Python*
* *https://github.com/fluentpython/example-code*


## 接口：从协议到抽象基类

抽象类表示接口。 ---Bjarne Stroustrup C++之父

抽象基类和描述符、元类一样，是用与构建工具。

**接口是实现特定角色方法的集合。协议是接口，但不是正式的(只由文档和约定定义)，因此协议不能像接口那样施加限制。**

### Python喜欢序列

Sequence抽象基类

![](https://img2018.cnblogs.com/blog/1615330/201903/1615330-20190306102500676-1503644188.png)

In [74]:
class Foo():
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]

In [75]:
Foo()[1]

10

In [76]:
for i in Foo()[:]:
    print(i)

0
10
20


In [77]:
10 in Foo()

True

```
虽然没有__iter__方法，但是Foo实例是可迭代的，因为发现有__getitem__方法时，Python会调用它，传入从0开始的整数索引。尽管没有实现__contain__方法，但是Python足够智能，能够迭代Foo实例，因此能够使用in云算法：Python会全部迭代，看看有没有指定的元素。

如果没有__iter__和__contain__方法，Python会调用__getitem__方法，设法让迭代和in运算法可用。
```

![](https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=2668458762,2133731349&fm=15&gp=0.jpg)

### 标准库中的抽象基类

* *https://www.python.org/dev/peps/pep-3119/*
* *https://docs.python.org/3/library/abc.html*

#### collections.abc模块中的抽象基类

![](https://bugs.python.org/file47357/base.png)

```
Iterable、Container和Sized

    各个集合应该继承这三个抽象类，或者至少实现兼容协议。Iterable通过__iter__方法支持迭代，Container通过__contains__方法支持in运算符，Sized通过__len__方法支持len()函数。
    
Sequence、Mapping和Set

    这三个主要是不可变集合类型，而且各自都有可变的子类。

Callable和hashable

    这两个抽象基类和集合没有太大的关系，只不过因为collections.abc是标准库定义抽象基类的第一个模块，而它们又太重要了，因此，它们才放到collections.abc模块中。
    
Iterator

    它是Iterable的子类。
        
```

#### 抽象基类的数字塔

numbers包定义的数字塔，其中Number是最顶端的超类，随后是Complex子类

* Number
* Complex
* Real
* Rational
* Integral

### 定义并使用抽象基类

In [78]:
import abc

In [81]:
class Tombola(abc.ABC):
    @abc.abstractmethod
    def load(self, iterable):
        """从可迭代对象中添加元素"""
        
    @abc.abstractmethod
    def pick(self):
        """随机删除元素，然后将其返回"""
        
    @abc.abstractmethod
    def loaded(self):
        """如果至少由一个元素，返回True，否则返回False"""
        return bool(self.inspect())
        
    def inspect(self):
        """返回一个有序元组，由当前元素构成。"""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                 break
        self.load(items)
        return tuple(sorted(items))

### 抽象类示例

In [65]:
from abc import ABC, abstractmethod

In [70]:
class Strategy(ABC):
    @abstractmethod
    def discount(self, order): pass

In [71]:
class CustomStrategy(Strategy):
    def discount(self, order): pass

In [72]:
Strategy()

TypeError: Can't instantiate abstract class Strategy with abstract methods discount

> 抽象类不能被实例化

In [73]:
CustomStrategy()

#### 实现原理

```
Implementation: The @abstractmethod decorator sets the function attribute __isabstractmethod__ to the value True. The ABCMeta.__new__ method computes the type attribute __abstractmethods__ as the set of all method names that have an __isabstractmethod__ attribute whose value is true. It does this by combining the __abstractmethods__ attributes of the base classes, adding the names of all methods in the new class dict that have a true __isabstractmethod__ attribute, and removing the names of all methods in the new class dict that don't have a true __isabstractmethod__ attribute. If the resulting __abstractmethods__ set is non-empty, the class is considered abstract, and attempts to instantiate it will raise TypeError. (If this were implemented in CPython, an internal flag Py_TPFLAGS_ABSTRACT could be used to speed up this check [6].)
```

## 对象引用、可变性和垃圾回收

### 元组的相对不变性

元组与多数Python集合(列表、字典、集合等)一样，保存的是对象的引用。如果引用元素是可变的，即使元组本身不可变，元素依然可变。元组的不可变性其实是指tuple数据结构的的物理内容可不变，与引用的对象无关。

### 默认做浅复制

*http://pythontutor.com/*

构造方法或[:] 做的是浅复制。

In [37]:
l1 = [3, [66, 55, 44], (7, 8, 9)] 
l2 = list(l1) # ➊
l1.append(100) # ➋
l1[1].remove(55) # ➌ 
print('l1:', l1) 
print('l2:', l2)
l2[1] += [33, 22] # ➍ 
l2[2] += (10, 11) # ➎ 
print('l1:', l1)
print('l2:', l2)

l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]
l1: [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2: [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]


```
➊ l2 是 l1 的浅复制副本。此时的状态如图 8-3 所示。
➋ 把 100 追加到 l1 中，对 l2 没有影响。
➌ 把内部列表 l1[1] 中的 55 删除。这对 l2 有影响，因为 l2[1] 绑定的列表与 l1[1] 是同一个。
➍ 对可变的对象来说，如 l2[1] 引用的列表，+= 运算符就地修改列表。这次修改在 l1[1]中也有体现，因为它是 l2[1] 的别名。
➎ 对元组来说，+= 运算符创建一个新元组，然后重新绑定给变量 l2[2]。这等同于 l2[2] =l2[2] + (10, 11)。现在，l1 和 l2 中最后位置上的元组不是同一个对象。如图 8-4 所示。
```

In [45]:
l = (1, 2, 3)

In [44]:
id(l)

4404228512

In [46]:
l += (4, 5)

In [47]:
id(l)

4418397968

> l地址改变，指向不同的对象

In [49]:
l2 = [1, 2, 3]

In [50]:
id(l2)

4418936496

In [51]:
l2 += [4, 5]

In [52]:
id(l2)

4418936496

> l2地址未变，仍然指向同一个对象

### 函数的参数作为引用时

函数可能会改变参数传入的可变对象。

In [53]:
def f(a, b):
    a += b
    return a

In [54]:
x = 1
y = 2

f(x, y), x, y

(3, 1, 2)

> x 未改变

In [55]:
a = [1, 2]
b = [3, 4]
f(a, b), a, b

([1, 2, 3, 4], [1, 2, 3, 4], [3, 4])

> a值变了

In [56]:
t = (10, 20)
u = (30, 40)
f(t, u), t, u

((10, 20, 30, 40), (10, 20), (30, 40))

> t未改变

#### 不要使用可变类型作为参数的默认值

```python

>>> bus1 = HauntedBus(['Alice', 'Bill']) 
>>> bus1.passengers
['Alice', 'Bill']
>>> bus1.pick('Charlie')
>>> bus1.drop('Alice')
>>> bus1.passengers ➊ 
['Bill', 'Charlie']
>>> bus2 = HauntedBus() ➋ 
>>> bus2.pick('Carrie')
>>> bus2.passengers
['Carrie']
>>> bus3 = HauntedBus() ➌
>>> bus3.passengers ➍
['Carrie']
>>> bus3.pick('Dave')
>>> bus2.passengers ➎
['Carrie', 'Dave']
>>> bus2.passengers is bus3.passengers ➏ True
>>> bus1.passengers ➐
['Bill', 'Charlie']

➊ 目前没什么问题，bus1 没有出现异常。
➋ 一开始，bus2 是空的，因此把默认的空列表赋值给 self.passengers。 ➌ bus3 一开始也是空的，因此还是赋值默认的列表。
➍ 但是默认列表不为空!
➎ 登上 bus3 的 Dave 出现在 bus2 中。
➏ 问题是，bus2.passengers 和 bus3.passengers 指代同一个列表。
➐ 但 bus1.passengers 是不同的列表。

```

#### 防御可变参数

```python
class TwilightBus:
"""让乘客销声匿迹的校车"""
    def __init__(self, passengers=None): 
        if passengers is None:
            self.passengers = [] ➊ 
        else:
        self.passengers = passengers ➋ 
    def pick(self, name):
        self.passengers.append(name)
    def drop(self, name):
        self.passengers.remove(name) ➌

➊ 这里谨慎处理，当 passengers 为 None 时，创建一个新的空列表。
➋ 然而，这个赋值语句把 self.passengers 变成 passengers 的别名，而后者是传给 __
init__ 方法的实参(即示例 8-14 中的 basketball_team)的别名。
➌ 在 self.passengers 上调用 .remove() 和 .append() 方法其实会修改传给构造方法的那
个列表。
```

### del和垃圾回收

**在CPython中，垃圾回收机制使用的主要算法是引用计数。**

### 弱引用

正式因为引用，对象才会在内存中存在。当对象的引用的计数为0后，垃圾回收程序会把对象销毁。

弱引用不会增加对象的引用计数。引用的目标对象称为所指对象(refrent)。因此，弱引用不会妨碍所引用对象被垃程序销毁。

**弱引用的局限性**

不是每个Python对象都可以作为弱引用的目标。基本的list和dict实例不能作为所指对象，但是它们的子类可以轻松解决这个问题。

set实例可以作为所指对象。

## 可迭代的对象、迭代器和生成器

迭代是数据处理的基石。扫描内存中放不下的数据集时，我要找到一种惰性获取数据的方式，即按需一次获取一个数据项。这就是迭代器(Iterator pattern)。

> 所有的生成器都是迭代器，因为生成器完全实现了迭代器接口。在Python社区中，大多数时候把迭代器和生成器当作同一个概念。

### Sentence类第1版：单词序列

In [82]:
import re

In [83]:
RE_WORD = re.compile('\w+')

In [84]:
class Sentence(object):
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)

In [86]:
s = Sentence('"The time has come," the Walrus said,')
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


**序列可以迭代的原因：iter函数**

```
解释器需要迭代对象时，会自动调用iter(x).

内置的iter函数有以下作用：

1. 检查对象是否实现了__iter__方法，如果实现了，就调用它，获取一个迭代器
2. 如果没有实现__iter__方法，但是实现了__getitem__方法，Python会创建一个迭代器，尝试按顺序(从0开始)获取元素
3. 如果尝试失败，Python会抛出TypeError异常
```

> 从Python 3.4开始，检查对象是否能迭代，最准确的方法是：调用iter(x)函数，如果不可迭代，再处理TypeError异常。

### 可迭代的对象和迭代器的对比

```
可迭代的对象

    使用iter内置函数可以获取迭代器对象。如果对象实现了能返回迭代器的__iter__方法，那么对象是可迭代的。序列都可以迭代；实现了__getitem__方法，而且参数是从零开始的索引，这种对象也可以迭代。
```

```python
class Iterable(metaclass=ABCMeta):
    __slots__ = ()

    @abstractmethod
    def __iter__(self):
        while False:
            yield None

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Iterable:
            if any("__iter__" in B.__dict__ for B in C.__mro__):
                return True
        return NotImplemented
```


```python
class Iterator(Iterable):
    __slots__ = ()
    
    @abstractmethod
    def __next__(self):
        'Return the next item from the iterator. When exhausted, raise StopIteration'
        raise StopIteration

    def __iter__(self):
        return self

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Iterator:
            if (any("__next__" in B.__dict__ for B in C.__mro__) and
                any("__iter__" in B.__dict__ for B in C.__mro__)):
                return True

        return NotImplemented
```

### Sentence类第2版：典型的迭代器

In [89]:
class SentenceIterator:
    def __init__(self, words):
        self.words = words
        self.index = 0
        
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
            
        self.index += 1
        return word
    
    def __iter__(self):
        return self

In [90]:
class Sentence(object):
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __iter__(self):
        return SentenceIterator(self.words)
    
    def __len__(self):
        return len(self.words)

In [91]:
s = Sentence('"The time has come," the Walrus said,')
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


```
可迭代的对象一定不能是自身的迭代器。也就是说，可迭代的对象必须实现了__iter__方法，但不能实现__next__方法。
另一方面，迭代器可以一直迭代。迭代器的__iter__方法应该返回本身。
```

### Sentence类第3版：生成器函数

使用生成器代替SetenceIterator类。

In [94]:
class Sentence(object):
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __iter__(self):
        for word in self.words:
            yield word
    
    def __len__(self):
        return len(self.words)

In [95]:
s = Sentence('"The time has come," the Walrus said,')
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


**生成器函数的工作原理**

只要Python函数定义体中有yield关键字，该函数就是生成器函数。调用生成器函数，就会返回一个生成器对象。

### Sentence第4版：惰性实现

设计Iterator接口时，考虑惰性：next(my_iterator)一次生成一个元素。其实：惰性求值(lazy evaluation)和及早求值(eager evaluation)是编程语言的理论方面的技术术语。

re.finditer函数是re.findall函数的惰性版本，返回的不是列表，而是一个生成器，按需生成re.MatchObject实例。如果有很多匹配项，re.finditer函数可以节省大量内存。

In [97]:
class Sentence(object):
    def __init__(self, text):
        self.text = text

    def __iter__(self):
        for match in RE_WORD.finditer(self.text):
            yield match.group()
    
    def __len__(self):
        return len(self.words)

In [98]:
s = Sentence('"The time has come," the Walrus said,')
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


### Sentence第5版：生成器表达式

生成器表达式可以理解为列表推到的惰性版本，不会迫切构建列表，而是返回一个生成器，按需惰性生成元素。

In [99]:
class Sentence(object):
    def __init__(self, text):
        self.text = text

    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))
    
    def __len__(self):
        return len(self.words)

In [100]:
s = Sentence('"The time has come," the Walrus said,')
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


### 标准库中的生成器函数

标准库提供了很多生成器，有的用于逐行迭代文本，还有出色的os.walk函数。

### Python3.3中新出现的句法：yield from

如何生成器函数需要产出另一个生成器生成的值，传统的解决办法是使用嵌套的for循环。

In [105]:
def chain(*iterables):
    for it in iterables:
        for i in it:
            yield i

In [106]:
list(chain('ABC', tuple(range(3))))

['A', 'B', 'C', 0, 1, 2]

In [107]:
def chain(*iterables):
    for it in iterables:
        yield from it

In [108]:
list(chain('ABC', tuple(range(3))))

['A', 'B', 'C', 0, 1, 2]

```
yield from i完全替代了内部的for循环。除了替代循环外，yield from还会创建通道把内层生成器直接与外层生成器的客户端连接起来。把生成器 当作协程使用时，这个通道特别重要，不仅能为客户端代码生成值，还能使用客户端代码提供的值。
```

### 深入分析iter函数

iter函数还有一个鲜为人知的用法：传入两个参数，一个参数是可调用的对象，一个是哨兵，当可调用的对象返回这个值时，迭代器抛出StopIteration异常。

### 延伸阅读

**生成器和迭代器对比**

```
1. 接口。Python的迭代器协议定义了两个方法：__next__和__iter__。生成器对象实现了这两个方法。因此，从这方面看，所有的生成器都是迭代器
2. 实现方式。生成器可以有两种编写方式：一种是yield关键字，一种是生成器表达式
3. 概念。迭代器用于遍历集合，从中产生元素。生成器可能无需遍历集合就能生成值
```

## 协程

* *http://seriously.dontusethiscode.com/2013/05/01/greedy-coroutine.html*
* *http://dabeaz.com/coroutines/*
* *http://dabeaz.com/coroutines/Coroutines.pdf*
* *http://flupy.org/resources/yield-from.pdf*

从句法上看，协程和生成器很类似，都是定义体内包含yield关键字的函数。可是在协程中，yield通常出现在表达式的右边，例如：num = yield，可以产出值，也可以不产出。如果yield关键字后面没有表达式，那么生成器产出None。协程可能会从调用方接收数据，不过调用方把数据提供给协程使用的.send(datam)方法，而不是next(datam)函数。通常调用方会把指送给协程。

不管数据如何流动，yield都是一种流程控制工具，使用它可以实现协作式多任务：协程可以把控制器让步给中心调度程序，从而激活其他协程。

### 生成器如何进化成协程

生成器的调用方可以使用.send()方法发送数据，发送的数据会成为生成器函数中yield表达式的值。因此，生成器可以作为协程。协程是指一个过程，这个过程和调用方协作，产出由调用方提供的值。

除了.send()方法，PEP342还提供了.throw()和.close()方法：前者的作用是让调用方抛出异常，后者的作用是终止生成器。

**迭代器vs协程**

* Generator produce data for iteration
* Coroutine are consumers of data

### 用协程的生成器的基本行为

In [109]:
def simple_coroutine():
    print('-> coroutine started')
    x = yield
    print('-> coroutine received: ', x)

In [110]:
cor = simple_coroutine()

In [111]:
cor

<generator object simple_coroutine at 0x107aaf450>

In [112]:
next(cor)

-> coroutine started


In [113]:
cor.send(42)

-> coroutine received:  42


StopIteration: 

协程有4种状态

* GEN_CREATED 等待开始执行
* GEN_RUNNING 解释器正在执行
* GEN_SUSPENDED 在yield表达式处暂停
* GEN_CLOSE 执行结束

In [114]:
import inspect

In [115]:
inspect.getgeneratorstate(cor)

'GEN_CLOSED'

### 使用协程实现计算移动平均值

In [127]:
def averager():
    count = 0
    total = 0
    average = None
    while True:
        num = yield average
        count += 1
        total += num

        average = total / count

In [132]:
avg = averager()
next(avg)

In [133]:
avg.send(10)

10.0

In [134]:
avg.send(20)

15.0

### 预激协程的装饰器

In [135]:
import functools

In [136]:
def coroutine(func):
    def wrapper(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen

    return wrapper

In [137]:
@coroutine
def averager():
    count = 0
    total = 0
    average = None
    while True:
        num = yield average
        count += 1
        total += num

        average = total / count

In [138]:
avg = averager()

In [139]:
avg.send(10)

10.0

In [140]:
avg.send(20)

15.0

### 终止协程和异常处理

```
gen.throw(exc_type, [, exc_val [, traceback]])

    致使生成器在暂停的yield表达式处抛出指定的异常。如果生成器处理了异常，代码会前执行到下一个yield表达式，而产出的值会成为调用gen.throw()方法得到的返回值。如果生成器没有处理异常，异常会向上冒泡，传到调用方的上下文中。
    
gen.close()

    致使生成器在暂停的yield表达式处抛出GeneratorExit异常。如果生成器没有处理这个异常，或者抛出了StopIteration异常，调用方不会报错。如果收到GeneratorExit异常，生成器一定不能产出值，否则解释器会抛出RuntimeError异常。生成器抛出的其他异常会向上冒泡，传给调用方。
```

In [194]:
class DemoException(Exception): pass


def demo_exc_handling():
    print('coroutine startd')
    while True:
        try:
            x = yield
        except DemoException:
            print('DemoException handled. continue')
        else:
            print('coroutine received: ', x)

激活和关闭demo_exc_handling，没有异常

In [195]:
cor = demo_exc_handling()
next(cor)
cor.send(10)
cor.send(20)
cor.close()
inspect.getgeneratorstate(cor)

coroutine startd
coroutine received:  10
coroutine received:  20


'GEN_CLOSED'

把DemoException传入generator不会导致gen终止

In [196]:
cor = demo_exc_handling()
next(cor)
cor.send(10)
cor.send(20)
cor.throw(DemoException)
inspect.getgeneratorstate(cor)

coroutine startd
coroutine received:  10
coroutine received:  20
DemoException handled. continue


'GEN_SUSPENDED'

无法处理的异常，协程会终止

In [197]:
cor = demo_exc_handling()
next(cor)
cor.send(10)
cor.send(20)
cor.throw(ZeroDivisionError)

coroutine startd
coroutine received:  10
coroutine received:  20


ZeroDivisionError: 

In [158]:
inspect.getgeneratorstate(cor)

'GEN_CLOSED'

### 让协程返回值

In [159]:
import collections

In [161]:
Result = collections.namedtuple('Result', 'count average')

def averager():
    count = 0
    total = 0
    average = None
    while True:
        num = yield average
        
        if num is None:
            break

        count += 1
        total += num

        average = total / count

    return Result(count, average)

In [164]:
avg = averager()
next(avg)
avg.send(10)
avg.send(20)
avg.send(None)

StopIteration: Result(count=2, average=15.0)

In [166]:
try:
    avg = averager()
    next(avg)
    avg.send(10)
    avg.send(20)
    avg.send(None)
except StopIteration as exc:
    result = exc.value
    print(result)

Result(count=2, average=15.0)


### 使用yield from

In [167]:
def chain(*iterables):
    for it in iterables:
        yield from it

In [168]:
list(chain('ABC', range(3)))

['A', 'B', 'C', 0, 1, 2]

yield from x表达式对x对象所做的第一件事就是调用iter(x)，从中获取迭代器，因此，x可以是任何可迭代的对象。

yield from的主要功能是打开双向通道，把最外层的调用方与最内层的子生成器连接起来，这样二者可以直接发送和产出值，还可以直接传入异常，而不用在中间的协程中添加大量处理异常的代码。

```
委派生成器
    
    包含yield from <iterable>表达式的生成函数
    
子生成器
   
    从yield from表达式中<iterable>部分获取的生成器
    
调用方

    调用委派生成器的客户端代码。在不同的语境下，我会使用客户端代替调用方。
```

> ```引入yield from结构的目的是为了支持实现__next__、send、close和throw方法的生成器```

![](https://timgsa.baidu.com/timg?image&quality=80&size=b9999_10000&sec=1581597460476&di=4fb9817999071a314d79fd8b7e05f6ec&imgtype=0&src=http%3A%2F%2Fpic1.zhimg.com%2Fv2-8a35d3f97cfce1c0edc39d5c53f88440_b.jpg)

In [210]:
Result = collections.namedtuple('Result', 'count average')

def averager():
    count = 0
    total = 0
    average = None
    while True:
        num = yield 
        
        if num is None:
            break

        count += 1
        total += num

        average = total / count

    return Result(count, average)

In [211]:
def grouper(results, key):
    while True:
        results[key] = yield from averager()

In [212]:
def main(data):
    results = {}
    
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)
        group.send(None)
        
    print(results)

In [213]:
data = {
    'girls;kg': [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m': [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43]
}

In [214]:
main(data)

{'girls;kg': Result(count=10, average=42.040000000000006), 'girls;m': Result(count=10, average=1.4279999999999997)}


### yield from的意义

把迭代器当作生成器使用，相当于把子生成器的定义内联在yield from表达式中。此外，子生成器可以执行return语句，返回一个值，而返回的值会成为yield from表达式的值。

PEP380

```
* 子生成器产出的值直接委派给生成器的调用方
* 使用send()方法发给委派生成器的值都直接传给子生成器。如果发送的是None，那么会调用子生成器的__next__方法，如果发送的不是None，那么会调用子生成器的send()方法。如果调用的方法抛出StopIteration异常，那么委派生成器恢复运行。任何其他异常都会向上冒泡，传递给委派生成器
* 生成器退出时，生成器(或子生成器)中的return expr表达式会触发StopIteration(expr)异常退出
* yield from表达式的值是子生成器终止时传递给StopIteration异常的第一个参数
```

yield from结构的另外两个特性与异常和终止有关。

```
* 传入委派生成器的异常，除了GeneratorExit之外都传给子生成器的throw方法。如果调用throw方法时抛出StopIterator异常，委派生成器恢复运行。StopIteration之外的异常会向上冒泡，传递给委派生成器。
* 如果把GeneratorExit异常传入委派生成器，或者在委派生成器上调用close()方法，那么在子生成器调用close()方法，如果它有的话。如果调用close()方法导致异常抛出，那么异常向上冒泡，传递给委派生成器；否则，委派生成器抛出GeneratorExit()异常。
```

yield from 实现伪代码

```python
_i =  iter(EXPR)

try:
    _y = next(_i)
except StopIteration as _e:
    _r = _e.value
else:
    while 1:
        _s = yield _y
        try:
            _y = _i.send(_s)
        except StopIteration as _e:
            _r = _e.value
            break
            
RESULT = _r
```

* _i 迭代器，子生成器
* _y 产出的值，子生成器产出的值
* _s 发送的值，调用方发给委派生成器的值，这个值会转给子生成器
* _r 结果，最终的结果(即子生成器运行结束后yield from表达式的值)
* _e 异常对象

## 使用future处理并发

futuer是一种对象，表示异步执行的操作，这个概念的作用很大，是concurrent.futures模块和asyncio包的基础。

### 期物future

从Python3.4起，标准库中有两个名为Future的类：concurrent.futures.Future和asyncio.Future。Future类实例表示可能已经完成或者尚未完成的延迟计算。这和Tornado框架中Future类似。

通常情况下，我们不能自己创建Future，而只能由并发框架实例化。

客户端代码不应该改变Future的状态，并发框架在Future表示的延迟计算结束后，会改变Future的状态，而我们无法控制计算何时结束。

两种Future都有.done()方法，这个方法不阻塞，返回的是布尔值，指明Future链接的可调用对象是否已经执行。客户端通常不会询问Future是否运行结束，而是等待通知。因此，两个Future类都有.add_done_callback()方法：这个方法只有一个参数，类型是可调用的对象，Future运行结束时，会调用指定的可调用对象。

此外，还有.result()方法。在Future运行结束调用后，这个方法在两个Future类的作用相同：返回可调用对象的结果，或者抛出执行可调用对象抛出的异常。

**GIL几乎对I/O密集型应用无害**

### 阻塞型I/O和GIL

CPython解释器本身不是线程安全的，因此有GIL全局解释锁(GIL)，一次只允使用一个线程执行python字节码。因此，一个Python线程通常不能同时使用多个CPU核心。

**标准库所有执行阻塞I/O操作的函数，在等待系统操作返回结果时都会释放GIL。这意味着在Python语言层次上可以使用多线程，而I/O密集型Python程序能从中受益：一个Python线程等待网络响应时，阻塞型I/O函数会释放GIL，再运行一个线程。**

> Python标准库中所有的阻塞I/O都会释放GIL，允许其他线程执行。 time.sleep()函数也会释放GIL。因此，尽管有GIL，Python多线程程序还是能在I/O密集型应用中发挥作用。

### 使用concurrent.futures模块启动进程

concurrent.futures模块实现的是真正的并行计算，因为它使用ProcessPoolExecuter类把工作分给多个Python进程处理。

## 使用asyncio包处理并发

```
并发是指一次处理多件事。
并行是指一次做多件事。
二者不同，但是有联系。
一个关于结构，一个关于执行。
并发用于指定方案，用来解决可能并行的问题。

--Rob Pike Go语言的创造者之一
```

真正的并行需要多个核心。实际上大多数过程都是并发处理，而不是并行处理。计算机始终运行者100多个进程，确保每个进程都有机会取得进展，不过CPU同时做的事情不能超过4件。

**Concurrency Is Not Parrallesim**

### 线程和协程的对比

#### asyncio.Future：故意不阻塞

在asyncio包中，BaseEventLoop.create_task()方法接收一个协程，排定它的运行时间，然后返回一个asyncio.Task实例--也就是asyncio.Future类的实例，因为Task是Future的子类，用于包装协程。

aysncio.Future类提供了.done()、.add_done_callback()和.result()等方法。

asyncio.Future类的.result()方法没有参数，因此不能指定超时时间。此外，如果调用.result()方法时期Future还没有执行完毕，那么.result()方法不会阻塞去等待结果，而是抛出asyncio.InvalidStateError异常。

aysncio.Future对象的结果通常使用yield from，从中产出结果。

使用yield from处理Future，等待Future运行完毕这一步无需我们关系，而且不会阻塞事件循环，因为在asyncio包中，yield from的作用把控制权还给事件循环。

总之，因为aysncio.Future类的目的是与yield from一起使用，所以通常不需要使用一下方法：

* 无需调用add_done_callback()，因为可以直接把在Future运行结束后执行的操作放在协程中yield from my_future表达式后面。这是协程一大优势：协程是可以暂停和恢复的函数
* 无需调用result()，因为yield from从Future中产出的值就是结果，例如：result = yield from my_future()

当然，有时也需要使用done()、add_done_callback()和result()方法，但是一般情况下，asyncio.Future对象由yield from驱动，而不是靠调用这些方法。

#### 从Future、任务和协程中产出

对于协程来说，获取task的方式主要有两种：

```
asyncio.async(coro_or_future, loop=None)

    这个函数统一了协程和Future。
    
BaseEventLoop.create_task(coro)

    这个方法排定协程的执行时间，返回一个asyncio.Task对象。
```


### 使用asyncio和aiohttp包下载

```python

import aiohttp  # <1>

from flags import BASE_URL, save_flag, show, main  # <2>


@asyncio.coroutine  # <3>
def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = yield from aiohttp.request('GET', url)  # <4>
    image = yield from resp.read()  # <5>
    return image


@asyncio.coroutine
def download_one(cc):  # <6>
    image = yield from get_flag(cc)  # <7>
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc


def download_many(cc_list):
    loop = asyncio.get_event_loop()  # <8>
    to_do = [download_one(cc) for cc in sorted(cc_list)]  # <9>
    wait_coro = asyncio.wait(to_do)  # <10>
    res, _ = loop.run_until_complete(wait_coro)  # <11>
    loop.close() # <12>

    return len(res)

```

```python
coroutine asyncio.wait(aws, *, loop=None, timeout=None, return_when=ALL_COMPLETED)
```

asyncio.wait是一个协程，等待给它的所有协程运行完毕后结束。wait()协程的参数是由协程或Future构成可迭代对象；wait会分别把协程包装进一个Task对象。最终的结果是，wait处理所有的对象都通过某种方式变成Future类的实例。wait是协程函数，因此返回的是协程或生成器对象；wait_coro变量中存储的正是这个对象。为了驱动协程，我们把协程传给loop.run_until_complete方法。

### 避免阻塞调用

有两种方法能避免阻塞调用终止整个应用程序：

* 在单独的线程中运行各个阻塞操作
* 把每个阻塞型操作换成非阻塞的异步调用

为了降低内存消耗，通常使用回调实现异步调用。当然，只有异步应用程序底层的事件循环能依靠基础设置的中断、线程、轮训和后台进程等，确保多个并发请求能取得进展并最终完成，这样，才能使用回调。

把生成器当作协程是异步编程的另一种方式。对于事件循环来说，调用回调和暂停的协程上调用.send()方法效果差不多。各个暂停的的协程是消耗内存，但是比线程消耗内存小。

### 从回调到Future和Coroutine

使用回调的一个很大问题就是“回调地狱”，不断嵌套回调。

使用协程和yield from结构实现异步编程，无需使用回调。

```python
@asyncio.coroutine
def three_stages(request1):
    response1 = yield from api_call1(request1) # 第一步
    request2 = step1(response1)
    response2 = yield from api_call2(request2) # 第二步
    request3 = step2(response2)
    response3 = yield from api_call3(request3) # 第三步
    step3(response3)

```

## 动态属性和特性

在Python中，数据的属性和处理数据的方法统称属性(attribute)。其实，方法是可调用的属性。

Python提供了丰富的API，用于控制属性的访问权限，以及实现动态属性。使用点号访问属性时，Python解释器会调用`__getattr__`方法计算属性。

动态创建属性是一种元编程，框架的作者经常这么做。

### 使用动态属性

```

我们通常把__init__成为构造方法，其实，用于构建实例的是特殊的方法__new__：这个类方法，必须返回一个实例。返回的实例会作为第一个参数传给__init__方法。调用__init__方法要传入实例，禁止返回任何值。所以__init__方法是初始化方法，真正的构造方法是__new__方法。
```

### 特性全解析

虽然内置的property经常用作装饰器，但它其实是一个类。

```python
property(fget=None, fset=None, fdel=None, doc=None)
```

### 处理属性的重要属性和函数

```
__class__ 

    对象所属类的引用， obj.__class__与type(obj)的作用相同。

dir

    列出对象的大都数属性。
    
getattr(object, name[, default])

    从对象中获取name字符串对应的属性。获取的属性可能来自对象所属的类或超类。如果没有指定的属性，getattr函数可以抛出AtrributeError异常，或者返回default参数值
    
hasattr(object, name)

    object对象是否有名字为name的属性
    
setattr(object, name, value)

    把object对象指定的属性的值设为value。这个函数可能创建一个新的属性，或者覆盖原来的属性。
    
vars([object])

    返回对象的__dict__属性。如果没指定参数，那么vars()函数的作用和locals()函数一样：返回表示作用域的字典。

```

#### 处理属性的特殊方法

```
__delattr__(self, name)

    只要使用del语句删除属性，就会调用这个方法。例如，del obj.attr语句。

__dir__(self)

    dir(obj)会触发Class.__dir__(obj)方法。
    
__getattr__(self, name)

    获取指定属性。
    
__getattribute__(self, name)

    尝试获取指定的属性时，总会调用这个方法，不过，寻找的是特殊属性或特殊方法时除外。点号与getattr和hasttr内置函数触发这个方法。调用__getattribute__方法且抛出AttributeError异常时，才会调用__getattr__方法。
    
__setattr__(self, name, value)

    设置属性值会调用这个方法。点号和setattr内置函数会触发这个方法。
```

## 属性描述符

```
学会描述符之后，不仅有更多的工具可以使用，还会对Python的运作方式有更深的理解，并由衷赞叹Python设计的优雅。 

--Raymond Hettinger Python核心开发者和专家
```

**描述符是对多个属性运用相同的方式存取逻辑的一种方式。** 例如，Django ORM和SQL Alchemy等ORM中的字段就是描述符，把数据库的记录中字段里的数据与Python对象的属性对应起来。

```
描述符是特定协议的类，这个协议包括__get__、__set__和__del__方法。property类实现了完整的描述符协议。
```

描述符是Python的独有特征，不仅在应用层中使用，在语言基础设施中也有用到。Python的classmethod和staticmethod装饰器的实现方式就是描述符。

### 覆盖型和非覆盖型的描述符对比

```
定义了__get__和__set__方法的典型覆盖型描述符
定义了__set__，而没有定义__get__的覆盖描述符
定义了__get__，没有定义__set__的非覆盖描述符
```

#### 覆盖型描述符

```
实现了__set__方法的描述符属于覆盖性描述符
```

#### 没有`__get__`方法的覆盖型描述符

没有实现`__get__`方法的描述符，获取的是描述符实例。

#### 非覆盖描述符

没有实现`__set__`方法的描述符是非覆盖描述符。

### 方法是描述符

在类中定义的函数属于绑定方法(bound method)，因为用户定义的函数都有`__get__`方法，所以依附到类上时，就相当于描述符。

函数没有实现`__set__`方法，因此是非覆盖描述符。

### 描述符使用建议

* 使用特性以保持简单
* 只读描述符必须有`__set__`方法，`__set__`抛出异常
* 用于验证的描述符可以只有`__set__`方法
* 仅有`__get__`方法的描述符可以实现高效缓存

## 类元编程

类元编程是指在运行时创建类或定制类的技术。在Python中，类是一等对象，因此在任何时候都可以使用函数创建新类，无需使用class关键字。元类是类元编程高级工具：使用元类可以创建具有某种特质的全新类，例如我们见过的抽象基类。

### 元类基础知识

**根据Python对象模型，类是对象，因此类肯定是另外某个类的实例。默认情况下，Python中的类是type类的实例。也就是说，type是大多数内置类和用户自定义类的元类。**

### 元类的特殊方法`__prepare__`

```
type构造方法以及元类的__new__和__init__方法都会收到要计算类的定义体，形式是名称到到属性的映射。然而，默认情况下，那个映射是个字典；也就是说，元类或类妆神器获得映射时，属性在类定义体中的顺序已经丢失了。

为了解决这个问题，Python3引入了特殊的方法：__prepare__。这个方法只在元类中有用，而且必须声明为类方法。解释器调用__new__方法之前，会先调用__prepare__方法，使用定义体中的属性创建映射。
```

### 类作为对象

类的属性

```
cls.__bases__
    
    由类的基类组成的元组。
    
cls.__qualname__

    Python3.3新引入的属性，其值是类或函数限定的名称，从模块的全局作用域到类的点分路基。
    
cls.__subclasses__

    返回一个列表，包含类的直接子类。这个方法的实现是弱引用，防止在超类和子类之间出现循环引用。这个方法返回的是内存中现存的子类。
    
cls.mro()

    类的方法解析顺序。
```

## Python Reference

## Python Data Types

* None
* Number
    * Int
    * Float
    * Boolean
* Sequence
    * String
    * List
    * Tuple
* Set
* Mapping
    * Dict
* Generator
* Class

### Python Metaclass

*https://mengyangyang.org/2017/07/18/python-metaclass/*

元类是创建类的类。元类是type类的派生类，通过重载type类的`__new__`和`__init__`方法，重新定义类创建协议，来实现类创建的定制化。

(对象)实例是通过类创建，类是通过type类创建，元类是type类的派生类。

* 类型(自定义)是通过type类或type派生元类创建
* 元类是type类的派生类
* 用户自定义类是type类的实例
* 用户自定义类可以生成自己的实例


```python
class = type(classname, superclasses, attributedict)
type.__new__(meta, classname, superclasses, attributedict)
type.__init__(cls, classname, superclasses, attributedict)
```

In [4]:
class Metaclass(type):
    def __new__(meta, classname, superclasses, attributedict):
        return type(classname, superclasses, attributedict)
    def __init__(cls, classname, superclasses, attributedict):
        pass

In [6]:
class Dummy(object, metaclass=Metaclass):
    pass

### Python Descriptor

* *https://mengyangyang.org/2017/07/10/python-descriptor/*
* *https://docs.python.org/2/reference/datamodel.html#descriptors*
* *https://docs.python.org/2/howto/descriptor.html*


如果一个对象定义了以下任意方法，这个对象就是一个描述符。给描述符下个定义，描述符就是绑定了行为属性的对象。

```python
object.__get__(self, instance, owner)
object.__set__(self, instance, value)
object.__delete__(self, instance)
```

```属性访问的默认行为就是从一个对象字典中获取、设置和删除属性。比如，a.x首先会搜索a.__dict__['x']，其次type(a).__dict__['x']，最后所有type(a)的元类。如果要查找的值一个包含描述器方法的对象，Python会用调用描述器方法代替默认行为。```

#### Descriptor Protocol


```python
object.__get__(self, ins, _type=None)
object.__set__(self, ins, value)
object.__del__(self, ins)
```

```
如果一个对象包含上面任意一个方法，就可以被看作是一个描述符。如果一个对象定义了__get__和__set__两个方法，该对象可以被看作一个数据描述符，如果一个对象只定义了__get__，该对象就是non-data描述符。
```

**数据描述符与非数据描述符的区别在于，描述符与对象实例entry调用优先级。如果一个实例的字典有一个entry和数据描述符的名字相同，数据描述符的调用的优先级高。如果一个实例有一个entry和非数据描述符的名字相同个，实例entry的调用的优先级高。**

#### Built-in Descriptor

In [16]:
class C(object):
    def getx(self): return self.__x
    def setx(self, value): self.__x = value
    def delx(self): del self.__x
    x = property(getx, setx, delx, "I'm the 'x' property.")


class Property(object):
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

#### `staticmethod` 和`classmethod`

In [18]:
class StaticMethod(object):
    "Emulate PyStaticMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f

    def __get__(self, obj, objtype=None):
        return self.f

In [19]:
class ClassMethod(object):
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f

    def __get__(self, obj, klass=None):
        if klass is None:
            klass = type(obj)
        def newfunc(*args):
            return self.f(klass, *args)
        return newfunc

### Python Context Manager

上下管理器是一个对象，定义了执行with语句时需要创建的上下文。

```python
context manager的__enter__()和__exit__()方法分别在进入、退出with语句时被调用。
object.__enter__(self)
object.__exit__(self, exc_type, exc_value, traceback)
```

```python
with_stmt ::=  "with" with_item ("," with_item)* ":" suite
with_item ::=  expression ["as" target]
```

with语句执行数据流：

1. 评估上下文表达式是否包含上下文管理器
2. 加载上下文管理器的__exit__方法
3. 调用上下文管理的__enter__方法
4. 如果target包含在with语句中，将__enter__方法的返回值赋给target
5. 执行suite
6. 调用__exit__()方法


#### Context lib

In [51]:
class GeneratorContextManager(object):
    """Helper for @contextmanager decorator."""

    def __init__(self, gen):
        self.gen = gen

    def __enter__(self):
        try:
            return self.gen.next()
        except StopIteration:
            raise RuntimeError("generator didn't yield")

    def __exit__(self, type, value, traceback):
        if type is None:
            try:
                self.gen.next()
            except StopIteration:
                return
            else:
                raise RuntimeError("generator didn't stop")
        else:
            if value is None:
                # Need to force instantiation so we can reliably
                # tell if we get the same exception back
                value = type()
            try:
                self.gen.throw(type, value, traceback)
                raise RuntimeError("generator didn't stop after throw()")
            except StopIteration as exc:
                # Suppress the exception *unless* it's the same exception that
                # was passed to throw().  This prevents a StopIteration
                # raised inside the "with" statement from being suppressed
                return exc is not value
            except:
                # only re-raise if it's *not* the exception that was
                # passed to throw(), because __exit__() must not raise
                # an exception unless __exit__() itself failed.  But throw()
                # has to raise the exception to signal propagation, so this
                # fixes the impedance mismatch between the throw() protocol
                # and the __exit__() protocol.
                #
                if sys.exc_info()[1] is not value:
                    raise

In [52]:
def contextmanager(func):
    @wraps(func)
    def helper(*args, **kwds):
        return GeneratorContextManager(func(*args, **kwds))
    return helper

In [53]:
from contextlib import contextmanager

In [55]:
@contextmanager
def tag(name):
    print('<%s>' % name)
    yield
    print('<%s>' % name)

In [56]:
with tag('h1'):
    print('websit')

<h1>
websit
<h1>


### Python Decorator

装饰器是一种设计模型（结构型模式），可以动态给一个对象添加一些额外的职责，而不用修改该对象的任何code。

装饰器优点：

* 比静态继承更灵活。与静态继承相比，装饰器可以灵活向对象添加额外的责任
* 避免在层次结构高层的类有特多的特征。 可以定义一个简单的类，通过装饰器给他逐渐添加功能。


In [71]:
def decorator(f):
    def wrapper(*args, **kwargs):
        print('before call %s' % f.__name__)
        result = f(*args, **kwargs)
        print('after call %s' % f.__name__)
        return result
    return wrapper

In [72]:
@decorator
def foo(): pass

In [74]:
foo()

before call foo
after call foo


#### 函数签名

In [4]:
import functools

In [5]:
def decorator(f):
    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        return f(*args, **kwargs)
    return wrapper

In [6]:
@decorator
def foo(): pass

In [7]:
foo.__name__

'foo'

#### 带参数的装饰器

In [13]:
def decorator_with_param(params, **optional_params):
    def decorator(f):
        @functools.wraps(f)
        def wrapper(*args, **kwargs):
            print('decorator')
            return f(*args, **kwargs)
        return wrapper

    if callable(params):
        return decorator(params)

    return decorator

In [16]:
@decorator_with_param('params')
def foo(): pass

In [17]:
foo()

decorator


#### 装饰器什么时候运行

装饰器在加载Python模块的时候运行。

#### 闭包

闭包延伸了作用域的函数，能够访问定义体之外的变量。

#### nonlocal

In [18]:
def make_average():
    count = 0
    total = 0
    def average(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        
        return total / count
    
    return average

In [20]:
avg = make_average()

In [21]:
avg(10)

10.0

In [22]:
avg(20)

15.0

#### Python内置的装饰器

* `functools.lru_cache`

### Python WSGI

WSGI(Web Server Gateway Interface) Web服务网关接口。

WSGI的目的替代CGI。CGI进程（类似Python解释器）针对每个请求进行创建，完成请求后退出。如果应程序接收树钱个请求，创建大量的语言解释器进程就会很快导致服务器宕机。

**其目标在Web服务器与Web框架层之间提供一个通用的API标准，减少之间互操作性，并形成统一的调用方式。**

### WSGI Application



根据WSGI的定义，其应用是可调用的对象，其参数固定为两个：

* 含有服务器环境变量的字典
* 可调用的对象， 该对象使用HTTP状态码和会返回客户端的HTTP头来初始化响应


In [121]:
def simple_wsgi_app(environment, start_response):
    status = '200 OK'
    headers = {'Content-Type': 'text/plain'}
    start_response(status, headers)
    return ['Hello world!']

```
environment 包含一些环境变量，如HTTP_HOST, HTTP_USER, HTTP_AGENT, SERVER_PROTOCOL等。‘
start_response()是一个可调用的对象，必须在应用执行，生成最终发送回客户端的响应
```

In [124]:
def start_response(status, response_headers, exc_info=None):
    if exc_info:
        try:
            if headers_sent:
                reraise(*exc_info)
        finally:
            exc_info = None
    elif headers_set:
        raise AssertionError('Headers already set')
    headers_set[:] = [status, response_headers]
    return write

**Django WSGI application**

```python
class WSGIHandler(base.BaseHandler):
    request_class = WSGIRequest

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.load_middleware()

    def __call__(self, environ, start_response):
        set_script_prefix(get_script_name(environ))
        signals.request_started.send(sender=self.__class__, environ=environ)
        request = self.request_class(environ)
        response = self.get_response(request)

        response._handler_class = self.__class__

        status = '%d %s' % (response.status_code, response.reason_phrase)
        response_headers = [
            *response.items(),
            *(('Set-Cookie', c.output(header='')) for c in response.cookies.values()),
        ]
        start_response(status, response_headers)
        if getattr(response, 'file_to_stream', None) is not None and environ.get('wsgi.file_wrapper'):
            response = environ['wsgi.file_wrapper'](response.file_to_stream, response.block_size)
        return response

```

#### WSGI Server

```python

import os, sys

def run_with_cgi(application):

    environ = dict(os.environ.items())
    environ['wsgi.input']        = sys.stdin
    environ['wsgi.errors']       = sys.stderr
    environ['wsgi.version']      = (1, 0)
    environ['wsgi.multithread']  = False
    environ['wsgi.multiprocess'] = True
    environ['wsgi.run_once']     = True

    if environ.get('HTTPS', 'off') in ('on', '1'):
        environ['wsgi.url_scheme'] = 'https'
    else:
        environ['wsgi.url_scheme'] = 'http'

    headers_set = []
    headers_sent = []

    def write(data):
        if not headers_set:
             raise AssertionError("write() before start_response()")

        elif not headers_sent:
             # Before the first output, send the stored headers
             status, response_headers = headers_sent[:] = headers_set
             sys.stdout.write('Status: %s\r\n' % status)
             for header in response_headers:
                 sys.stdout.write('%s: %s\r\n' % header)
             sys.stdout.write('\r\n')

        sys.stdout.write(data)
        sys.stdout.flush()

    def start_response(status, response_headers, exc_info=None):
        if exc_info:
            try:
                if headers_sent:
                    # Re-raise original exception if headers sent
                    raise exc_info[0], exc_info[1], exc_info[2]
            finally:
                exc_info = None     # avoid dangling circular ref
        elif headers_set:
            raise AssertionError("Headers already set!")

        headers_set[:] = [status, response_headers]
        return write

    result = application(environ, start_response)
    try:
        for data in result:
            if data:    # don't send headers until body appears
                write(data)
        if not headers_sent:
            write('')   # send headers now if body was empty
    finally:
        if hasattr(result, 'close'):
            result.close()
```

## Python tips

### Python 新式类与旧式类

旧式类的MRO是深度优先遍历，新式类的MRO是广度优先遍历，Python3全部是新式类。

### Python `__new__` vs `__init__`

*https://docs.python.org/3/reference/datamodel.html#basic-customization*


* `__new__` 创建实例
* `__init__` 初始化实例
* `__new__` 返回对象实例
* `__init__` 返回None
* `__new__` 第一个参数是cls
* `__init__` 第一个参数self


### Python Singleton

In [152]:
class Singleton(object):
    _instance = None
    def __new__(cls,*args, **kwargs):
        if cls._instance is None:
            cls._instance = object.__new__(cls, *args, **kwargs)
        return cls._instance

In [153]:
class Foo(Singleton): pass

In [154]:
Foo() is Foo()

True

In [145]:
def singleton(cls):
    _instances = {}
    def inner(*args, **kwargs):
        if cls not in _instances:
            _instances[cls] = cls(*args, **kwargs)
        return _instances[cls]
    return inner

In [146]:
@singleton
class Foo(object): pass

In [147]:
Foo() is Foo()

True

### Python 作用域

LEGB

* Local
* Enclosing Locals
* Global
* Built-in

### Python GIL

线程全局锁(Global Interpreter Lock),即Python为了保证线程安全而采取的独立线程运行的限制,说白了就是一个核只能在同一时间运行一个线程.对于io密集型任务，python的多线程起到作用，但对于cpu密集型任务，python的多线程几乎占不到任何优势，还有可能因为争夺资源而变慢。

### Python2 vs Python3

*http://chenqx.github.io/2014/11/10/Key-differences-between-Python-2-7-x-and-Python-3-x/*

* print
* 整除
* unicode
* xrange
* raise exception

### Python is vs ==

* `is`判断是否为同一个对象
* `==`判断值是否相等

> == 运算法比较的是两个对象的值，is比较对象的标识。is运算符比==速度快，因为它不能重载，所以Python不用寻找并调用特殊方法，而是直接比较两个对象的ID。a == b是语法糖，等同于a.`__eq__`(b)，继承自object的`__eq__`方法，

### Python tuple

* tuple是不可更改的
* tuple也经常用于记录，例如nametuple

### 格式化显示

In [59]:
format(2/3, '0.1%')

'66.7%'

In [60]:
import datetime

In [61]:
format(datetime.datetime.now(), '%H:%M:%S')

'23:45:59'

### 强类型和弱类型

如果一门语言很少隐式转换类型，说明它是强类型语言;如果经常这么做，说明它 是弱类型语言。Java、C++ 和 Python 是强类型语言。PHP、JavaScript 和 Perl 是弱类型语言。

### 静态类型和动态类型


在编译时检查类型的语言是静态类型语言，在运行时检查类型的语言是动态类型 语言。静态类型需要声明类型(有些现代语言使用类型推导避免部分类型声明)。 Fortran 和 Lisp 是最早的两门语言，现在仍在使用，它们分别是静态类型语言和动态类型语言。
